# Open Data Loop

Captures, in an open loop, camera data.

Modeled off of: https://sfeister.github.io/sidekick-epics-docs/demos/OpenLoopHRR/

## Import Modules

In [1]:
import numpy as np
import epics # 'epics' is part of pyepics
from epics import PV
from time import sleep
import matplotlib.pyplot as plt

In [2]:
pvroi1 = PV("CAM:ROI1:DATA.VAL", form="time")
pvroi2 = PV("CAM:ROI2:DATA.VAL", form="time")
leds = PV("LEDS:data.VAL", form="time")

In [3]:
print(pvroi1.value)
print(pvroi2.value)
print(leds.value)

TRIG:388,R:71,G:121,B:243
TRIG:388,R:56,G:36,B:126
6291,200,200,200,200,200,


In [4]:
epics.caput("CAM:TRIGCNT", 12555)

1

In [5]:
def ledparse(datastr):
    """ Parse the LED brightness data string, which comes in the format '123441,251,12,201,12,156,' where it goes 'TRIG, LED0, LED1, LED2, LED3, LED4' """
    vals = np.array(datastr.split(','))[:-1]
    trig = np.int64(vals[0])
    brigs = np.array(np.uint8(vals[1:]))
    return trig, brigs

In [6]:
def camparse(datastr):
    """ Parse the RGB-color data string, which comes in the format 'TRIG:123441,R:251,G:12,B:201' """
    vals = [elem.split(":")[1] for elem in datastr.split(',')]
    trig = np.int64(vals[0])
    rgb = np.array(np.uint8(vals[1:]))
    return trig + 752, rgb

In [7]:
print(camparse(pvroi1.value))
print(camparse(pvroi2.value))
print(ledparse(leds.value))

(1140, array([ 71, 121, 243], dtype=uint8))
(1140, array([ 56,  36, 126], dtype=uint8))
(6291, array([200, 200, 200, 200, 200], dtype=uint8))


In [8]:
def dataCallback(pvname=None, value=None, **kw):
    global isfresh
    isfresh = True

epics.caput("PULSEGEN:reprate", 0.25)
pvroi1.add_callback(dataCallback)

NSTEPS = 10
for i in range(NSTEPS):
    isfresh = False
    while not isfresh:
        sleep(0.01)
    trig1, rgb1 = camparse(pvroi1.value)
    trig2, rgb2 = camparse(pvroi2.value)
    trig3, brigs = ledparse(leds.value)
    print(trig1, trig2, trig3)
    print(rgb1, rgb2, brigs)
    for j in range(len(brigs)):
        epics.caput('LEDS:CH{}:brig.VAL'.format(j), i*50)
    #print('ROI1: ' + pvroi1.value)
    #print('ROI2: ' + pvroi2.value)

pvroi1.clear_callbacks()

1141 1141 6291
[ 71 122 244] [ 56  36 126] [200 200 200 200 200]
1142 1141 6292
[17 20 20] [ 56  36 126] [200 200 200 200 200]
1143 1143 6294
[ 36  53 158] [ 58  36 139] [50 50 50 50 50]
1144 1144 6294
[ 48  80 208] [ 57  37 136] [50 50 50 50 50]
1145 1145 6296
[ 66  97 226] [ 58  33 126] [150 150 150 150 150]
1146 1146 6296
[ 83 111 237] [ 58  31 115] [150 150 150 150 150]
1147 1147 6298
[ 90 133 250] [ 56  34 119] [250 250 250 250 250]
1148 1148 6298
[ 88 137 252] [ 55  34 119] [250 250 250 250 250]
1149 1149 6300
[ 93 132 250] [ 57  32 115] [255 255 255 255 255]
1150 1150 6300
[ 88 138 252] [ 55  35 120] [255 255 255 255 255]


In [9]:
print(pvroi1.value)

TRIG:398,R:88,G:138,B:252


In [10]:
pvroi1.clear_callbacks()

In [11]:
def testCallback(pvname=None, value=None, **kw):
    print('New PVROI Data!')
    print(value)

pvroi1.add_callback(testCallback)
sleep(10)
pvroi1.clear_callbacks()

New PVROI Data!
TRIG:399,R:87,G:138,B:252
New PVROI Data!
TRIG:400,R:88,G:139,B:252
